This is script will figure out travel times from different runs

In [41]:
import argparse
import pandas

# declare some column names
travelTime = 'travelTime'
startTime = 'startTime'
endTime = 'endTime'
id = 'personId'
tripNumber = 'tripNo'
mode = 'mainMode'

def parse_args(args):
    parser = argparse.ArgumentParser()
    parser.add_argument('-f', '--files', nargs='+', required=True)
    parser.add_argument('-n', '--names', nargs='+', required=True)
    return parser.parse_args(args)

def read_csv(listOfFiles, listOfNames):
    
    assert(len(listOfFiles) == len(listOfNames)), 'number of files and scenario names should be equal but was: (' + str(len(listOfFiles)) + '/' + str(len(listOfNames)) + ')'
    
    result = []

    for file, name in zip(listOfFiles, listOfNames):
        scenario = pandas.read_csv(file, usecols=[id, tripNumber, startTime, endTime, mode])
        scenario.set_index([id, tripNumber], inplace=True)
        scenario.sort_index(inplace=True)
        scenario[travelTime] = scenario[endTime] - scenario[startTime]
        result.append(scenario)
    return result

def copy_travel_times(dataFrame, names):
    
    result = pandas.DataFrame()
    for name in names:
        result[get_column_name(travelTime, name)] = dataFrame[get_column_name(travelTime, name)]
        result[get_column_name(mode, name)] = dataFrame[get_column_name(mode, name)]
    return result
        
def get_column_name(column, scenario):
    
    return column + '_' + scenario


In [42]:
dataFolder = "C:\\Users\\Janekdererste\\Desktop\\nemo_analysis\\"
args = parse_args([
    '-f', dataFolder + 'base-case.csv', dataFolder + 'bike-highways.csv',
    '-n', 'Base Case', 'Bike Highways' 
])

dataFrames = read_csv(args.files, args.names)
data = pandas.concat(dataFrames, keys=args.names, names=['RunId', id, tripNumber])
print(data.head(10))

                           startTime  endTime mainMode  travelTime
RunId     personId tripNo                                         
Base Case 54804601 0         27000.0  31106.0      car      4106.0
                   1         35220.0  37119.0      car      1899.0
                   2         37659.0  38525.0      car       866.0
                   3         40500.0  41473.0      car       973.0
                   4         41653.0  42352.0      car       699.0
                   5         42592.0  45155.0      car      2563.0
          54822301 0         20820.0  23278.0      car      2458.0
                   1         46800.0  49348.0      car      2548.0
                   2         50940.0  52882.0      car      1942.0
          54847601 0         18720.0  21370.0      car      2650.0


In [52]:
# select the mode and the travel time
travelTimes = data[[travelTime, mode]]
print(travelTimes.unstack(level='RunId').head(30))

                travelTime                mainMode              
RunId            Base Case Bike Highways Base Case Bike Highways
personId tripNo                                                 
54804601 0          4106.0        4601.0       car           car
         1          1899.0        2600.0       car           car
         2           866.0         977.0       car           car
         3           973.0        1431.0       car           car
         4           699.0         659.0       car           car
         5          2563.0        2791.0       car           car
54822301 0          2458.0        2496.0       car           car
         1          2548.0        1344.0       car           car
         2          1942.0        1573.0       car           car
54847601 0          2650.0        2605.0       car           car
         1          1615.0        1944.0       car           car
         2          4848.0        4114.0       car           car
54882301 0          2027.

In [ ]:
# group by mode and by run id
grouped = travelTimes.groupby([mode, 'RunId']).mean()
# unstack the multi-index into two columns for both run ids and then plot
grouped.unstack().plot.bar(title='Travel Times')

                                   fromX         fromY            toX  \
          personId tripNo                                               
Base Case 54804601 0       379020.264258  5.705633e+06  388197.224919   
                   1       388197.224919  5.714843e+06  387835.177407   
                   2       387835.177407  5.710075e+06  390006.181976   
                   3       390006.181976  5.710844e+06  393812.335988   
                   4       393812.335988  5.709468e+06  392948.319484   
                   5       392948.319484  5.710825e+06  379020.264258   
          54822301 0       374369.800063  5.702648e+06  383465.963142   
                   1       383465.963142  5.720068e+06  375596.005378   
                   2       375596.005378  5.714855e+06  374369.800063   
          54847601 0       378560.218434  5.701469e+06  407716.500440   

                                    toY  startTime  endTime      distance  \
          personId tripNo                     